# Train a sensor processing model using a Convolutional Variational Autoencoder 

Using the Julian-8897-Conv-VAE-PyTorch implementation to train a sensor processing model based on convolutional variational autoencoder. 

The parameters of the training are described by an experiment run of type "sensorprocessing_conv_vae". The result of runing the code in this notebook is the model files that are stored in the experiment directory. 

As the model files will have unpredictable date-time dependent names, after running a satisfactory model, the mode name and directory will need to be copied to the experiment/run yaml file, in the model_subdir and model_checkpoint fields.


In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import shutil
import pprint
from demonstration.demonstration import Demonstration, get_simple_transform


# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# At some point in the development, this hack was necessary for some reason. 
# It seems that as of Feb 2025, the code runs on Windows and Linux without it.
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml


In [2]:
# If it is set to true, no actual copying will be done
dry_run = False

# Specify and load the experiment
experiment = "sensorprocessing_conv_vae"
run = "sp_vae_128" 
# run = "sp_vae_128_300epochs" 
# run = "sp_vae_256" 
# run = "sp_vae_256_300epochs" 
exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)

***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_conv_vae\sp_vae_128_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: sensorprocessing_conv_vae/sp_vae_128 successfully loaded
Experiment:
    class: ConvVaeSensorProcessing
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\sp_vae_128
    epochs: 100
    exp_run_sys_indep_file: C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\sensorprocessing_conv_vae\sp_vae_128.yaml
    experiment_name: sensorprocessing_conv_vae
    image_size:
    - 64
    - 64
    json_template_name: conv-vae-config-default.json
    latent_size: 128
    model_dir: models
    model_name: VAE_Robot
    run_name: sp_vae_128
    save_period: 5
    subrun_name: null
    training_data:
    - - random-both-cameras
      - '2025_03_08__14_15_53'
      - 

### Create the training data for the Conv-VAE

We collect the training data for the Conv-VAE by gathering all the pictures from all the demonstrations of a specific task. One can select the pictures by creating a specific task, and copy there all the relevant demonstrations. 

The collected pictures are put in a newly created training directory for the run:

```
$experiment\vae-training-data\Images\*.jpg
```

In [3]:
def copy_images_to_training_dir(exp, training_image_dir):
    """Copy all the images specified in the training_data field to the training directory."""
    count = 0
    transform = get_simple_transform()
    print("***Train-Conv-VAE***: Copying training images to training directory")
    for val in exp["training_data"]:
        run, demo_name, camera = val
        exp_demo = Config().get_experiment("demonstration", run)
        demo = Demonstration(exp_demo, demo_name)
        for i in range(demo.metadata["maxsteps"]):
            training_image_path = pathlib.Path(training_image_dir, f"train_{count:05d}.jpg")
            demo.write_image(i, training_image_path, camera=camera, transform=transform)
            count += 1
    print(f"***Train-Conv-VAE***: Copying training images to training directory done")


In [4]:

# Deciding on the location of the training data
training_data_dir = pathlib.Path(exp.data_dir(), exp["training_data_dir"])
training_image_dir = pathlib.Path(training_data_dir, "Images")
# We assume that if the directory, exists, it had been previously populated with images
if not training_image_dir.exists():
    training_image_dir.mkdir(exist_ok = False, parents=True)
    copy_images_to_training_dir(exp, training_image_dir=training_image_dir)
else:
    print("***Train-Conv-VAE***: There are already images in training image dir {training_image_dir}. Do not repeat the copying.")            


***Train-Conv-VAE***: There are already images in training image dir {training_image_dir}. Do not repeat the copying.


# Run the training

Actually run the training. This is done by creating the json-based configuration file of the Conv-VAE library with the parameters specified in the library. Then we call the code of the library to perform the training. 

In [5]:
# Create the vae configuration, based on the experiment
file = create_configured_vae_json(exp)
print(file)
vae_config = get_conv_vae_config(file)

# actually run the training
print(f'***Train-Conv-VAE***: Running the trainer from scratch for {vae_config["trainer"]["epochs"]} epochs')
exp.start_timer("training")
trainer = train(vae_config)
exp.end_timer("training")

C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\conv-vae-config-default.json
{'name': 'VAE_Robot', 'n_gpu': 1, 'arch': {'type': 'VanillaVAE', 'args': {'in_channels': 3, 'latent_dims': 128, 'flow': False}}, 'data_loader': {'type': 'CelebDataLoader', 'args': {'data_dir': 'c:\\Users\\lboloni\\Documents\\Code\\_TempData\\BerryPicker-experiments\\sensorprocessing_conv_vae\\sp_vae_128\\vae-training-data', 'batch_size': 64, 'shuffle': True, 'validation_split': 0.2, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.005, 'weight_decay': 0.0, 'amsgrad': True}}, 'loss': 'elbo_loss', 'metrics': [], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamma': 0.1}}, 'trainer': {'epochs': 100, 'save_dir': 'c:\\Users\\lboloni\\Documents\\Code\\_TempData\\BerryPicker-experiments\\sensorprocessing_conv_vae\\sp_vae_128\\models', 'save_period': 5, 'verbosity': 2, 'monitor': 'min val_loss', 'early_stop': 10, 'tensorboard': True}}
c:\Users\lboloni\Docum

C:\Users\lboloni\Documents\Code\_Checkouts\Conv-VAE-PyTorch\utils\util.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._data.total[key] += value * n
C:\Users\lboloni\Documents\Code\_Checkouts\Conv-VAE-PyTorch\utils\util.py:60: Futu

***Timer*** training finished in 868.058821 seconds


In [6]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)
# 
trainer.train_metrics._data
# trainer.valid_metrics._data

,total,counts,average
loss,72789.980713,22,3308.635487


__Important__ After the training finished, in order to use the resulting system, one need to edit the run file (eg: vae_01.yaml) and enter into it the location of the checkpoint. This is the content printed by the code cell below

__Important__ Right now, it is set up such that only every 5. epoch is checkmarked. 

In [7]:

print(f"model_subdir: '{trainer.checkpoint_dir.name}'")
print(f"model_checkpoint: 'checkpoint-epoch{trainer.epochs}.pth'")


model_subdir: '0615_091131'
model_checkpoint: 'checkpoint-epoch100.pth'


In [8]:
if "exp_run_sys_dep_file" in exp:
    print(f'The text above to be put into \n the system dependent experiment run file {exp["exp_run_sys_dep_file"]}')
else:
    print(f'As the system dependent experiment run file does not exist,\n the text can be put into the system independent file\n {exp["exp_run_sys_indep_file"]}')


As the system dependent experiment run file does not exist,
 the text can be put into the system independent file
 C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\sensorprocessing_conv_vae\sp_vae_128.yaml


In [9]:
checkpoint_path = pathlib.Path(trainer.checkpoint_dir, f"checkpoint-epoch{trainer.epochs}.pth")
model_target_path = pathlib.Path(exp.data_dir(), "model.pth")

json_path = pathlib.Path(trainer.checkpoint_dir, "config.json")
json_target_path = pathlib.Path(exp.data_dir(), "config.json")

if checkpoint_path.exists():
    print(f"***Train-Conv-VAE***: Copying the checkpoint from {checkpoint_path} to {model_target_path}")
    model_target_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(checkpoint_path, model_target_path)
    # target_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(json_path, json_target_path)
else:
    print(f"***Train-Conv-VAE***: The checkpoint file {checkpoint_path} does not exist. Cannot copy it to model.pth")

***Train-Conv-VAE***: Copying the checkpoint from c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\sp_vae_128\models\models\VAE_Robot\0615_091131\checkpoint-epoch100.pth to c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\sp_vae_128\model.pth


In [10]:
print(trainer.checkpoint_dir)
print(trainer.checkpoint_dir.exists())
print(checkpoint_path)
print(checkpoint_path.exists())

c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\sp_vae_128\models\models\VAE_Robot\0615_091131
True
c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\sp_vae_128\models\models\VAE_Robot\0615_091131\checkpoint-epoch100.pth
True
